In [1]:
# Untuk load data dari gdrive
from google.colab import drive
# Untuk pengolahan data
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path
from IPython.display import display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

# Load Dataset `tour`

In [2]:
url = 'https://drive.google.com/file/d/1DUDSMdnADerCl2PnlkddetEahHK1nF3L/view?usp=drive_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
maps = pd.read_csv(path)

In [3]:
maps

,Place_Id,Place_Name,Description,Category,Price,Rating,Coordinate,Latitude,Longitude,Address,City,Price_Grouped,Content_Text
0,85.0,Taman Pintar Yogyakarta,Taman Pintar Yogyakarta (bahasa Jawa: Hanacara...,Taman Hiburan,6000,4.5,"{'lat': -7.800671500000001, 'lng': 110.3676551}",-7.800671,110.367655,"Jalan Sriwedani, Kampung Ketandan, Prawirodirj...",Yogyakarta,50000,Taman Hiburan Yogyakarta 50000
1,86.0,Keraton Yogyakarta,Keraton Ngayogyakarta Hadiningrat atau Keraton...,Budaya,15000,4.6,"{'lat': -7.8052845, 'lng': 110.3642031}",-7.805284,110.364203,"Royal Palace of Yogyakarta, Jalan Suryoputran,...",Yogyakarta,50000,Budaya Yogyakarta 50000
2,87.0,Sindu Kusuma Edupark (SKE),Sindu Kusuma Edupark (SKE) merupakan sebuah de...,Taman Hiburan,20000,4.2,"{'lat': -7.767297300000001, 'lng': 110.3542486}",-7.767297,110.354249,"Sindu Kusuma Edupark, Jalan Jambon V, Kricak, ...",Sleman,50000,Taman Hiburan Sleman 50000
3,88.0,Museum Benteng Vredeburg Yogyakarta,Museum Benteng Vredeburg (bahasa Jawa: ???????...,Budaya,3000,4.6,"{'lat': -7.800201599999999, 'lng': 110.3663044}",-7.800202,110.366304,"Benteng Vredeburg, Jalan Margo Mulyo, Kampung ...",Yogyakarta,50000,Budaya Yogyakarta 50000
4,89.0,De Mata Museum Jogja,Museum De Mata merupakan salah satu museum yan...,Budaya,50000,4.4,"{'lat': -7.816315599999999, 'lng': 110.3871442}",-7.816316,110.387144,"XT Square, Jalan Veteran, Gambiran, Pandeyan, ...",Yogyakarta,50000,Budaya Yogyakarta 50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,206.0,Wisata Kaliurang,"Jogja selalu menarik untuk dikulik, terlebih t...",Cagar Alam,8000,4.4,"{'lat': -7.6120675, 'lng': 110.4205209}",-7.612068,110.420521,"Jalan Boyong, Ngipiksari, Kaliurang, Hargobina...",Sleman,50000,Cagar Alam Sleman 50000
122,207.0,Heha Sky View,HeHa Sky View adalah salah satu tempat wisata ...,Taman Hiburan,15000,4.4,"{'lat': -7.8496144, 'lng': 110.478324}",-7.849614,110.478324,"Resto HeHa sky view, Jalan Sindangsari, Patuk,...",Gunung Kidul,50000,Taman Hiburan Gunung Kidul 50000
123,208.0,Taman Sungai Mudal,"Taman Sungai Mudal, sebuah objek wisata alam t...",Cagar Alam,10000,4.6,"{'lat': -7.762813599999998, 'lng': 110.1161626}",-7.762814,110.116163,"Taman Sungai Mudal, Jalan Clapar, Bukit WIsata...",Kulonprogo,50000,Cagar Alam Kulonprogo 50000
124,209.0,Pantai Sanglen,Pantai Sanglen. Lokasinya berada di Desa Kemad...,Bahari,10000,4.5,"{'lat': -8.1367456, 'lng': 110.5716362}",-8.136746,110.571636,"Pantai Sangleng, Jalan Pantai Sepanjang, Kemad...",Gunung Kidul,50000,Bahari Gunung Kidul 50000


# Membuat Fungsi Scraping Link Lokasi

In [27]:
# Daftar tempat wisata
places = ['Taman Pintar Yogyakarta', 'Keraton Yogyakarta', 'Sindu Kusuma Edupark (SKE)', 'Museum Benteng Vredeburg Yogyakarta', 'De Mata Museum Jogja', 'Kampung Wisata Taman Sari', 'Situs Warungboto', 'Nol Kilometer Jl.Malioboro', 'Gembira Loka Zoo', 'Sumur Gumuling', 'Desa Wisata Sungai Code Jogja Kota', 'Alun Alun Selatan Yogyakarta', 'Monumen Yogya Kembali', 'Taman Pelangi Yogyakarta', 'Kampung Wisata Kadipaten', 'Taman Budaya Yogyakarta', 'Budaya', 'Kampung Wisata Sosro Menduran', 'Monumen Batik Yogyakarta', 'Tugu Pal Putih Jogja', 'Taman Hiburan', 'Tebing Breksi', 'Candi Donotirto', 'Kawasan Malioboro', 'Bangsal Pagelaran', 'Budaya', 'Embung Tambakboyo', 'Hutan Pinus Pengger', 'Monumen Serangan Umum 1 Maret', 'Puncak Pinus Becici', 'Bukit Bintang Yogyakarta', 'Taman Hiburan', 'Gedung Agung Yogyakarta', 'The Lost World Castle', 'Monumen Sanapati', 'Jurang Tembelan Kanigoro', 'The World Landmarks - Merapi Park Yogyakarta', 'Museum Sonobudoyo Unit I', 'Budaya', 'Lava Bantal', 'Bukit Panguk Kediwung', 'Candi Prambanan', 'Watu Goyang', 'Kampung Wisata Rejowinangun', 'Budaya', 'Kauman Pakualaman Yogyakarta', 'Budaya', 'Alun-alun Utara Keraton Yogyakarta', 'Pantai Goa Cemara', 'Blue Lagoon Jogja', 'Gumuk Pasir Parangkusumo', 'Bukit Lintang Sewu', 'Taman Hiburan', 'Studio Alam Gamplong', 'Watu Lumbung', 'Air Terjun Kedung Pedut', 'Puncak Kebun Buah Mangunan', 'Taman Hiburan', 'Desa Wisata Gamplong', 'Kawasan Wisata Sosrowijayan', 'Grojogan Watu Purbo Bangunrejo', 'Wisata Alam Kalibiru', 'Jogja Exotarium', 'Puncak Gunung Api Purba - Nglanggeran', 'Bendung Lepen', 'Bunker Kaliadem Merapi', 'Ledok Sambi', 'Seribu Batu Songgo Langit', 'Goa Jomblang', 'Desa Wisata Rumah Domes/Teletubbies', 'Bukit Wisata Pulepayung', 'Bentara Budaya Yogyakarta (BBY)', 'Goa Rancang Kencono', 'Goa Cerme', 'Wisata Kraton Jogja', 'Desa Wisata Kelor', 'Pantai Patihan', 'Pantai Kukup', 'Pantai Ngrawe (Mesra)', 'Pantai Ngobaran', 'Pantai Pulang Sawal', 'Pantai Baron', 'Pantai Pok Tunggal', 'Hutan Pinus Asri', 'Pasar Kebon Empring Bintaran', 'Bukit Paralayang, Watugupit', 'Kampung Wisata Dipowinatan', 'Watu Mabur Mangunan', 'Cagar Alam', 'Pintoe Langit Dahromo', 'Green Village Gedangsari', 'Taman Hiburan', 'Kebun Teh Nglinggo', 'Geoforest Watu Payung Turunan', 'Pantai Timang', 'Bahari', 'Puncak Segoro', 'Candi Borobudur', 'Candi Ijo', 'Budaya', 'Pantai Indrayanti', 'Pantai Nglambor', 'Pasar Beringharjo', 'Museum Ullen Sentalu', 'Budaya', 'Museum Gunung Merapi', 'Pantai Parangtritis', 'Goa Pindul', 'Candi Ratu Boko', 'Pantai Depok Jogja', 'Pantai Sundak', 'Hutan Mangrove Kulon Progo', 'Bahari', 'Jogja Bay Pirates Adventure Waterpark', 'Pantai Sepanjang', 'Pantai Krakal', 'Pantai Glagah', 'Pantai Siung', 'Pantai Sadranan', 'Pantai Nguluran', 'Pantai Samas', 'Pantai Drini', 'Air Terjun Sri Gethuk', 'Candi Sewu', 'Pantai Wediombo', 'Pantai Jogan', 'Pantai Ngrenehan', 'Pantai Jungwok', 'Pantai Greweng', 'Pantai Sedahan', 'Pantai Watu Kodok', 'Pantai Kesirat', 'Pantai Ngandong', 'Galaxy Waterpark Jogja', 'Desa Wisata Pulesari', 'Desa Wisata Tembi', 'Wisata Kaliurang', 'Heha Sky View', 'Taman Sungai Mudal', 'Pantai Sanglen', 'Pantai Congot']

# Setup Chrome driver
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

results = []

for place in places:
    driver.get("https://www.google.com/maps")
    time.sleep(2)

    search_box = driver.find_element(By.ID, "searchboxinput")
    search_box.clear()
    search_box.send_keys(place)
    search_box.send_keys(Keys.RETURN)

    time.sleep(5)  # beri waktu untuk load hasil

    url = driver.current_url
    print(f"{place} => {url}")
    results.append({"Place_Name": place, "Google_Maps_Link": url})

driver.quit()

# Simpan ke Excel
df = pd.DataFrame(results)
df.to_excel("hasil_google_maps_links.xlsx", index=False)
print("✅ Hasil disimpan ke hasil_google_maps_links.xlsx")


Taman Pintar Yogyakarta => https://www.google.com/maps/place/Taman+Pintar+Yogyakarta/@-7.8006715,110.3676551,17z/data=!3m1!4b1!4m6!3m5!1s0x2e7a57860f2fb9af:0xa9f6007074b62dd8!8m2!3d-7.8006715!4d110.3676551!16s%2Fm%2F0y6kq2z?entry=ttu&g_ep=EgoyMDI1MDUyNy4wIKXMDSoASAFQAw%3D%3D
Keraton Yogyakarta => https://www.google.com/maps/search/Keraton+Yogyakarta/@-7.8091125,110.3625054,15z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI1MDUyNy4wIKXMDSoASAFQAw%3D%3D
Sindu Kusuma Edupark (SKE) => https://www.google.com/maps/place/SKE+City+Park/@-7.7679241,110.3547164,17z/data=!3m1!4b1!4m6!3m5!1s0x2e7a584281200f8b:0xeed0884f4b2272a8!8m2!3d-7.7679241!4d110.3547164!16s%2Fg%2F11b6vc54pq?entry=ttu&g_ep=EgoyMDI1MDUyNy4wIKXMDSoASAFQAw%3D%3D
Museum Benteng Vredeburg Yogyakarta => https://www.google.com/maps/place/Vredeburg+Fort+Museum/@-7.800293,110.3661642,17z/data=!3m1!4b1!4m6!3m5!1s0x2e7a5788c0b3eecf:0xb9611ce0232a9ff8!8m2!3d-7.800293!4d110.3661642!16zL20vMGJyZmY4?entry=ttu&g_ep=EgoyMDI1MDUyNy4wIKXMDSoASAFQAw%3D%3D


In [28]:
# simpan ke csv
df.to_csv('hasil_google_maps_links.csv', index=False)


# Load dataset `hasil_google_maps_links`

In [4]:
url = 'https://drive.google.com/file/d/1UCXICU6J8F59wBawEpTWZHqimOn330Z4/view?usp=drive_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
link = pd.read_csv(path)

In [5]:
link

,Place_Name,Google_Maps_Link
0,Taman Pintar Yogyakarta,https://www.google.com/maps/place/Taman+Pintar...
1,Keraton Yogyakarta,https://www.google.com/maps/search/Keraton+Yog...
2,Sindu Kusuma Edupark (SKE),https://www.google.com/maps/place/SKE+City+Par...
3,Museum Benteng Vredeburg Yogyakarta,https://www.google.com/maps/place/Vredeburg+Fo...
4,De Mata Museum Jogja,https://www.google.com/maps/place/De+Mata+Tric...
...,...,...
136,Wisata Kaliurang,https://www.google.com/maps/search/Wisata+Kali...
137,Heha Sky View,https://www.google.com/maps/place/HeHa+Sky+Vie...
138,Taman Sungai Mudal,https://www.google.com/maps/place/Taman+Sungai...
139,Pantai Sanglen,https://www.google.com/maps/place/Pantai+Sangl...


# Cek dan Hapus Missing Value pada Place_Name

In [6]:
# Cek kolom Place_Name di data link yang beda dengan Place_Name di data maps
link[~link['Place_Name'].isin(maps['Place_Name'])]

,Place_Name,Google_Maps_Link
16,Budaya,https://www.google.com/maps/search/Budaya/@35....
20,Taman Hiburan,https://www.google.com/maps/search/Taman+Hibur...
25,Budaya,https://www.google.com/maps/search/Budaya/@35....
31,Taman Hiburan,https://www.google.com/maps/search/Taman+Hibur...
38,Budaya,https://www.google.com/maps/search/Budaya/@35....
44,Budaya,https://www.google.com/maps/search/Budaya/@36....
46,Budaya,https://www.google.com/maps/search/Budaya/@36....
52,Taman Hiburan,https://www.google.com/maps/search/Taman+Hibur...
57,Taman Hiburan,https://www.google.com/maps/search/Taman+Hibur...
88,Cagar Alam,https://www.google.com/maps/search/Cagar+Alam/...


In [7]:
# hapus
link = link[link['Place_Name'] != 'Taman Hiburan']

# Merge data berdasarkan Place_Name

In [8]:
# gabung data maps dan link menurut Place_Name maps, abaikan kapital
maps = maps.merge(link, on='Place_Name', how='left')

In [9]:
maps

,Place_Id,Place_Name,Description,Category,Price,Rating,Coordinate,Latitude,Longitude,Address,City,Price_Grouped,Content_Text,Google_Maps_Link
0,85.0,Taman Pintar Yogyakarta,Taman Pintar Yogyakarta (bahasa Jawa: Hanacara...,Taman Hiburan,6000,4.5,"{'lat': -7.800671500000001, 'lng': 110.3676551}",-7.800671,110.367655,"Jalan Sriwedani, Kampung Ketandan, Prawirodirj...",Yogyakarta,50000,Taman Hiburan Yogyakarta 50000,https://www.google.com/maps/place/Taman+Pintar...
1,86.0,Keraton Yogyakarta,Keraton Ngayogyakarta Hadiningrat atau Keraton...,Budaya,15000,4.6,"{'lat': -7.8052845, 'lng': 110.3642031}",-7.805284,110.364203,"Royal Palace of Yogyakarta, Jalan Suryoputran,...",Yogyakarta,50000,Budaya Yogyakarta 50000,https://www.google.com/maps/search/Keraton+Yog...
2,87.0,Sindu Kusuma Edupark (SKE),Sindu Kusuma Edupark (SKE) merupakan sebuah de...,Taman Hiburan,20000,4.2,"{'lat': -7.767297300000001, 'lng': 110.3542486}",-7.767297,110.354249,"Sindu Kusuma Edupark, Jalan Jambon V, Kricak, ...",Sleman,50000,Taman Hiburan Sleman 50000,https://www.google.com/maps/place/SKE+City+Par...
3,88.0,Museum Benteng Vredeburg Yogyakarta,Museum Benteng Vredeburg (bahasa Jawa: ???????...,Budaya,3000,4.6,"{'lat': -7.800201599999999, 'lng': 110.3663044}",-7.800202,110.366304,"Benteng Vredeburg, Jalan Margo Mulyo, Kampung ...",Yogyakarta,50000,Budaya Yogyakarta 50000,https://www.google.com/maps/place/Vredeburg+Fo...
4,89.0,De Mata Museum Jogja,Museum De Mata merupakan salah satu museum yan...,Budaya,50000,4.4,"{'lat': -7.816315599999999, 'lng': 110.3871442}",-7.816316,110.387144,"XT Square, Jalan Veteran, Gambiran, Pandeyan, ...",Yogyakarta,50000,Budaya Yogyakarta 50000,https://www.google.com/maps/place/De+Mata+Tric...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,206.0,Wisata Kaliurang,"Jogja selalu menarik untuk dikulik, terlebih t...",Cagar Alam,8000,4.4,"{'lat': -7.6120675, 'lng': 110.4205209}",-7.612068,110.420521,"Jalan Boyong, Ngipiksari, Kaliurang, Hargobina...",Sleman,50000,Cagar Alam Sleman 50000,https://www.google.com/maps/search/Wisata+Kali...
122,207.0,Heha Sky View,HeHa Sky View adalah salah satu tempat wisata ...,Taman Hiburan,15000,4.4,"{'lat': -7.8496144, 'lng': 110.478324}",-7.849614,110.478324,"Resto HeHa sky view, Jalan Sindangsari, Patuk,...",Gunung Kidul,50000,Taman Hiburan Gunung Kidul 50000,https://www.google.com/maps/place/HeHa+Sky+Vie...
123,208.0,Taman Sungai Mudal,"Taman Sungai Mudal, sebuah objek wisata alam t...",Cagar Alam,10000,4.6,"{'lat': -7.762813599999998, 'lng': 110.1161626}",-7.762814,110.116163,"Taman Sungai Mudal, Jalan Clapar, Bukit WIsata...",Kulonprogo,50000,Cagar Alam Kulonprogo 50000,https://www.google.com/maps/place/Taman+Sungai...
124,209.0,Pantai Sanglen,Pantai Sanglen. Lokasinya berada di Desa Kemad...,Bahari,10000,4.5,"{'lat': -8.1367456, 'lng': 110.5716362}",-8.136746,110.571636,"Pantai Sangleng, Jalan Pantai Sepanjang, Kemad...",Gunung Kidul,50000,Bahari Gunung Kidul 50000,https://www.google.com/maps/place/Pantai+Sangl...


In [11]:
# simpan maps ke excel
maps.to_excel('maps.xlsx', index=False)

# Mengambil Place_Name dan Google_Maps_Link

In [12]:
# Membuat dataset bernama gmaps, isinya Place_Name dan Google_Maps_Link
gmaps = maps[['Place_Name', 'Google_Maps_Link']]

In [13]:
# simpan ke csv dan excel
gmaps.to_csv('gmaps.csv', index=False)
gmaps.to_excel('gmaps.xlsx', index=False)